In [20]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import h5py

In [21]:
matplotlib.rc('xtick', labelsize=28)
matplotlib.rc('ytick', labelsize=28)
matplotlib.rcParams['font.size']=28

In [22]:
data_path = "/home/ryan/Data/"

In [88]:
def get_masses(sbh_pos, rbins, coord, mass_table):

    r = (coord[:,0] - sbh_pos[0])**2 + (coord[:,1] - sbh_pos[1])**2 + (coord[:,2] - sbh_pos[2])**2

    m_in = []
    for R in rbins:
        m_in.append(mass_table[r <= R].sum())
    return np.array(m_in)

In [97]:
snap_path = data_path + '/run_CDM_L3N256_HY/snap_007.hdf5'
fof_path = data_path + '/run_CDM_L3N256_HY/fof_subhalo_tab_007.hdf5'

nBins = 1000
N_partTypes = 6
massive_partTypes = [0, 1, 4, 5]

max_no_subhalos = 5
with h5py.File(fof_path, 'r') as s, h5py.File(snap_path, "r") as f:
    subhalo = s.get('Subhalo')
    group = s.get('Group')

    for subhalo_idx in range(max_no_subhalos):
        cm = subhalo['SubhaloCM'][subhalo_idx]
        max_rad = 3 * subhalo['SubhaloHalfmassRad'][subhalo_idx]
        nums = subhalo["SubhaloLenType"][subhalo_idx]
        group_idx = subhalo["SubhaloGrNr"][subhalo_idx]

        particles_before = np.zeros(N_partTypes).astype(int)
        if group_idx != 0:
            # find all particles in groups before this group
            particles_before += np.sum([ group['GroupLenType'][i] for i in range(group_idx) ], axis=0).astype(int)

        # which subhalo is this one within its group?
        first_in_group_idx = group["GroupFirstSub"][group_idx]
        # if -1, this is it
        if first_in_group_idx != -1:
            # get particles for the previous subhalos
            particles_before += np.sum([ subhalo['SubhaloLenType'][i] for i in range(first_in_group_idx, subhalo_idx) ], axis=0).astype(int)
            
        particles_this = subhalo['SubhaloLenType'][subhalo_idx]

        rbins = np.geomspace(max_rad / 30, max_rad, num=nBins)

        total_mass = np.zeros(nBins)
        for partType in massive_partTypes:

            coords = f.get(f"PartType{partType}/Coordinates")[()][particles_before[partType]:(particles_before[partType] + particles_this[partType])]
            if partType != 1:
                masses = f.get(f"PartType{partType}/Masses")[()][particles_before[partType]:(particles_before[partType] + particles_this[partType])]
            else:
                masses = f["Header"].attrs["MassTable"][1] * np.ones( particles_this[partType] ) 

            total_mass += get_masses(cm, rbins, coords, masses)

        
        densities = total_mass / ( (4/3) * np.pi * rbins**3 )

        fname = f'subhalo_{subhalo_idx}.txt'
        np.savetxt(fname, (rbins, densities))
